# &pi; in Julia

## Inline assembly instructions
(x86/x86_64 only)

Julia can call inline assembly via the `llvmcall` interface. The following snippet calls the `fldpi` instruction ("**f**loating point **l**oa**d** **pi**") which loads the constant &pi; onto the floating point register stack.

In [ ]:
function asm_pi()
    Base.llvmcall(
    """ %pi = call double asm "fldpi", "={st}"()
        ret double %pi""", 
    Float64, Tuple{})
end

In [ ]:
asm_pi()

We can look at the actual resulting code that is generated:

In [ ]:
@code_native asm_pi()

If you're wondering what the rest of these instructions are doing:

1. the `pushq` and `movq` adds to the [call stack frame](https://en.wikipedia.org/wiki/Call_stack).
2. `fldpi` pushes &pi; to the x87 floating point register stack
  - x87 is the older legacy floating point instruction set dating back to the original [Intel 8087 coprocessor](https://en.wikipedia.org/wiki/Intel_8087).
3. `fstpl` and `movsd` moves the value to the SSE floating point register `xmm0`
  - Julia, like most modern software, uses the newer SSE instruction set for its floating point operations. This also allows us to take advantage of things like [SIMD operations](https://en.wikipedia.org/wiki/SIMD).
4. `popq` and `retq` pops the call stack frame.